In [ ]:
import tensorflow as tf
from tensorflow import keras

import numpy as np

print tf.__version__

In [ ]:
imdb = keras.datasets.imdb
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

In [ ]:
A = {}
A['first'] = 27
A['ANOTHER'] = 28

In [ ]:
MYLIST  = [2,3,4,5]

In [ ]:
MYLIST.append(4)

In [ ]:
MYLIST

In [ ]:
m = NeuralNetwork()
m.train(X,y)
m.predict(X2)